In [1]:
import numpy as np
import pandas as pd

In [2]:
from my_packages.layers import Dense
from my_packages.neural_network import NN
from my_packages.environment import StockEnv
from my_packages.policy_train import PolicyTrain

In [3]:
%load_ext autoreload
%autoreload 2

# 1. Handling Data

In [4]:
api_query = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=1min&month=2025-02&outputsize=full&datatype=csv&apikey=6QP1RCVBQ5TCGE65"

In [5]:
df = pd.read_csv("data/IBM_feb_25.csv")
df.head()

,timestamp,open,high,low,close,volume
0,2025-02-28 19:59:00,252.5,252.5,252.50,252.50,1
1,2025-02-28 19:58:00,252.5,252.5,252.03,252.03,45
2,2025-02-28 19:53:00,252.5,252.5,252.50,252.50,116
3,2025-02-28 19:51:00,252.5,252.5,252.50,252.50,15
4,2025-02-28 19:49:00,252.5,252.5,252.50,252.50,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12388 entries, 0 to 12387
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  12388 non-null  object 
 1   open       12388 non-null  float64
 2   high       12388 non-null  float64
 3   low        12388 non-null  float64
 4   close      12388 non-null  float64
 5   volume     12388 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 580.8+ KB


In [7]:
df.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [8]:
# DATA ENGINEER WITH THE DATA I'M SKIPPING THE PART AS OF NOW 
# BE CAREFULL TO CHANGE THE INDEX OF RESPECTIVE DATA IN THE environment.py

In [9]:
df = df.iloc[:,1:]

In [10]:
X = df.to_numpy(dtype = float)

In [11]:
X.shape

(12388, 5)

# 2. Building model (Policy)

In [12]:
X[:5,:]

array([[252.5 , 252.5 , 252.5 , 252.5 ,   1.  ],
       [252.5 , 252.5 , 252.03, 252.03,  45.  ],
       [252.5 , 252.5 , 252.5 , 252.5 , 116.  ],
       [252.5 , 252.5 , 252.5 , 252.5 ,  15.  ],
       [252.5 , 252.5 , 252.5 , 252.5 ,   4.  ]])

In [13]:
data_scaler = np.ones(5) * 1e5
data_scaler

array([100000., 100000., 100000., 100000., 100000.])

In [14]:
X = X / data_scaler
X[:5,:]

array([[2.5250e-03, 2.5250e-03, 2.5250e-03, 2.5250e-03, 1.0000e-05],
       [2.5250e-03, 2.5250e-03, 2.5203e-03, 2.5203e-03, 4.5000e-04],
       [2.5250e-03, 2.5250e-03, 2.5250e-03, 2.5250e-03, 1.1600e-03],
       [2.5250e-03, 2.5250e-03, 2.5250e-03, 2.5250e-03, 1.5000e-04],
       [2.5250e-03, 2.5250e-03, 2.5250e-03, 2.5250e-03, 4.0000e-05]])

In [15]:
env = StockEnv()
env.set_env_data(X)
# STATES -> [OPEN , HIGH , LOW , CLOSE , VOLUME , AVG_SHARE_PRICE]

In [16]:
policy = NN([Dense(units=10, activation='relu'),
            Dense(units=3, activation='linear')],
            loss = 'mse')
policy.layers[0].init_layer(6)

In [17]:
env.num_shares

0

In [18]:
training_tool = PolicyTrain(policy, env, epsilon = 0.3, gamma = 0.9)

In [19]:
training_tool.episode_train(batch_size = 100)

Now the policy is ready and can be deserialized and stored and reused when required

# 3. Agent using policy

In [30]:
predict_data = X[10,:]
predict_data = np.append(predict_data,[10])
predict_data

array([2.524e-03, 2.524e-03, 2.524e-03, 2.524e-03, 1.000e-05, 1.000e+01])

In [31]:
policy.predict(predict_data)

array([[1.03758801, 1.38966191, 1.73104734]])